# Basic Classification and Object Detection on OpenVINO™ Integration with TensorFlow

## Introduction

This is a sample to showcase classification and object detection in images. For classification, InceptionV3 Model trained on ImageNet dataset is used and for object detection, Yolo V3 trained on COCO dateset used. This sample also showcases the performance and inference results using stock TensorFlow (Intel® CPUs) and OpenVINO™ Integration with TensorFlow (Intel® CPUs and Accelerators).

Accelerators refer to:
* Intel® integrated GPUs
* Intel® Neural Compute Stick with Movidius™ Vision Processing Units - referred as NCS2
* Intel® Vision Accelerator Design with 8 Intel Movidius™ MyriadX VPUs - referred as VAD-M or HDDL

First, we implement a standard TensorFlow classification application. The application loads the frozen TensorFlow graph, reads the input image and executes the inference on stock TensorFlow. Then, we activate OpenVINO™ integration with TensorFlow by adding two lines of code and re-run the inference. This sample will be running the inference on Intel® Core CPU only. To try other supported devices like Intel® HD Graphics GPU, Intel® Neural Compute Stick 2, and Intel® VAD-M, please use the more advanced samples.
  
## Install Prerequisites

Install TensorFlow, OpenVINO integration with TensorFlow, and additional prerequisities required for the demo. If any other version of TensorFlow is pre-installed on your system, you need to uninstall it first.

In [ ]:
!pip3 install tensorflow==2.4.1
!pip3 install openvino-tensorflow

!pip3 install numpy matplotlib opencv-python pillow

## Download Models, Images, and Labels

Download the models, images and labels required for the demo.

Note: You need to have python3-venv installed on your system before running the cell below. For Ubuntu 18.04, you can install it by running "apt-get install python3-venv".

In [ ]:
!mkdir data
!curl -L "https://storage.googleapis.com/download.tensorflow.org/models/inception_v3_2016_08_28_frozen.pb.tar.gz" | tar -C data -xz
!curl -L "https://raw.githubusercontent.com/openvinotoolkit/openvino_tensorflow/notebook_demo_branch/examples/data/grace_hopper.jpg" -o "data/grace_hopper.jpg"

!wget https://raw.githubusercontent.com/openvinotoolkit/openvino_tensorflow/notebook_demo_branch/examples/convert_yolov3_160.sh
!wget https://raw.githubusercontent.com/openvinotoolkit/openvino_tensorflow/notebook_demo_branch/examples/convert_yolov3.patch
!source convert_yolov3_160.sh

## Classification Sample

### Import Python Modules

In [ ]:
import tensorflow as tf
import os
import cv2
import numpy as np
import time
import matplotlib.pyplot as plt
from PIL import Image, ImageFont, ImageDraw

### Helper Functions

In [ ]:
def load_graph(model_file):
    graph = tf.Graph()
    graph_def = tf.compat.v1.GraphDef()
    assert os.path.exists(model_file), "Could not find model path"
    with open(model_file, "rb") as f:
        graph_def.ParseFromString(f.read())
    with graph.as_default():
        tf.import_graph_def(graph_def)

    return graph

def read_tensor_from_image_file(image_file,
                                input_height=299,
                                input_width=299,
                                input_mean=0,
                                input_std=255):
    assert os.path.exists(image_file), "Could not find image file path"
    image = cv2.imread(image_file)
    resized = cv2.resize(image, (input_height, input_width))
    img = cv2.cvtColor(resized, cv2.COLOR_BGR2RGB)
    resized_image = img.astype(np.float32)
    normalized_image = (resized_image - input_mean) / input_std
    result = np.expand_dims(normalized_image, 0)
    return result, image


def load_labels(label_file):
    label = []
    assert os.path.exists(label_file), "Could not find label file path"
    proto_as_ascii_lines = tf.io.gfile.GFile(label_file).readlines()
    for l in proto_as_ascii_lines:
        label.append(l.rstrip())
    return label

def print_classification_predictions(label_file, results):
    results = np.squeeze(results)
    with open(label_file) as file:
        labels = file.readlines()
    sorted_idx = np.argsort(results)
    print("Predictions:")
    for i in range(3):
        cls_index = sorted_idx[-(i+1)]
        label = labels[cls_index].strip()
        print("\t",label," (", "{:.8f}".format(results[cls_index]),")")

### Set Variables for Classification Sample

In [ ]:
file_name = "data/grace_hopper.jpg"
model_file = "data/inception_v3_2016_08_28_frozen.pb"
label_file = "data/imagenet_slim_labels.txt"
input_height = 299
input_width = 299
input_mean = 0
input_std = 255
input_layer = "input"
output_layer = "InceptionV3/Predictions/Reshape_1"
backend_name = "CPU"

### Load the Model and the Input Image

In [ ]:
graph = load_graph(model_file)

t, img = read_tensor_from_image_file(
         file_name,
         input_height=input_height,
         input_width=input_width,
         input_mean=input_mean,
         input_std=input_std)

### Import OpenVINO-TensorFlow Module and Set Backend

**DO NOT RUN** the cell below yet.
1. Skip this section and move on to the next section to run inference on stock TensorFlow. Note the inference latency.
2. Then, uncomment and run the cell when instructed below to activate OpenVINO integration with TensorFlow and run the inference again. Compare the inference latency with the previous one.

In [ ]:
# Import OpenVINO integration with TensorFlow and set the backend device
#import openvino_tensorflow as ovtf
#ovtf.set_backend('CPU')

### Run the Inference

In [ ]:
input_name = "import/" + input_layer
output_name = "import/" + output_layer
input_operation = graph.get_operation_by_name(input_name)
output_operation = graph.get_operation_by_name(output_name)

with tf.compat.v1.Session(graph=graph) as sess:
    # Warmup
    results = sess.run(output_operation.outputs[0],
                       {input_operation.outputs[0]: t})

    # Run
    start = time.time()
    results = sess.run(output_operation.outputs[0],
                       {input_operation.outputs[0]: t})
    elapsed = time.time() - start
    print('OVTF Inference time in ms: %.2f' % (elapsed * 1000))
    
    print_classification_predictions(label_file, results)
    
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()
    

### Enable/Disable OpenVINO integration with TensorFlow

Once you import OpenVINO-TensorFlow module, OpenVINO integration with TensorFlow will be enabled by default. If you need to manually enable/disable OpenVINO integration with TensorFlow, please use the API calls below and execute the inference again.

**To disable** OpenVINO ingtegration with TensorFlow, execute the cell below before running the inference.

In [ ]:
#ovtf.disable()

**To enable** OpenVINO ingtegration with TensorFlow, execute the cell below before running the inference.

In [ ]:
#ovtf.enable()

## Object Detection Sample

### Helper Functions

In [ ]:
def non_max_suppression(predictions_with_boxes,
                        confidence_threshold,
                        iou_threshold=0.4):
    conf_mask = np.expand_dims(
        (predictions_with_boxes[:, :, 4] > confidence_threshold), -1)
    predictions = predictions_with_boxes * conf_mask

    result = {}
    for i, image_pred in enumerate(predictions):
        shape = image_pred.shape
        non_zero_idxs = np.nonzero(image_pred)
        image_pred = image_pred[non_zero_idxs]
        image_pred = image_pred.reshape(-1, shape[-1])

        bbox_attrs = image_pred[:, :5]
        classes = image_pred[:, 5:]
        classes = np.argmax(classes, axis=-1)

        unique_classes = list(set(classes.reshape(-1)))

        for cls in unique_classes:
            cls_mask = classes == cls
            cls_boxes = bbox_attrs[np.nonzero(cls_mask)]
            cls_boxes = cls_boxes[cls_boxes[:, -1].argsort()[::-1]]
            cls_scores = cls_boxes[:, -1]
            cls_boxes = cls_boxes[:, :-1]

            while len(cls_boxes) > 0:
                box = cls_boxes[0]
                score = cls_scores[0]
                if cls not in result:
                    result[cls] = []
                result[cls].append((box, score))
                cls_boxes = cls_boxes[1:]
                # iou threshold check for overlapping boxes
                ious = np.array([iou(box, x) for x in cls_boxes])
                iou_mask = ious < iou_threshold
                cls_boxes = cls_boxes[np.nonzero(iou_mask)]
                cls_scores = cls_scores[np.nonzero(iou_mask)]

    return result

def convert_to_original_size(box, size, original_size, is_letter_box_image):
    if is_letter_box_image:
        box = box.reshape(2, 2)
        box[0, :] = letter_box_pos_to_original_pos(box[0, :], size,
                                                   original_size)
        box[1, :] = letter_box_pos_to_original_pos(box[1, :], size,
                                                   original_size)
    else:
        ratio = original_size / size
        box = box.reshape(2, 2) * ratio
    return list(box.reshape(-1))

def draw_boxes(boxes, img, cls_names, detection_size, is_letter_box_image):
    draw = ImageDraw.Draw(img)
    for cls, bboxs in boxes.items():
        #color = (256, 256, 256)
        color = (256, 0, 0)
        for box, score in bboxs:
            box = convert_to_original_size(box, np.array(detection_size),
                                           np.array(img.size),
                                           is_letter_box_image)
            draw.rectangle(box, outline=color, width=4)
            if os.path.isfile("/usr/share/fonts/truetype/dejavu/DejaVuSerif-Bold.ttf"):
                font = ImageFont.truetype("DejaVuSerif-Bold.ttf", int(img.size[0]/16))
            else:
                font = ImageFont.load_default()
            text_pos = box[:2]
            text_pos[0] += int(img.size[0]/32)
            text_pos[1] += int(img.size[1]/32)
            draw.text(
                text_pos,
                '{} {:.2f}%'.format(cls_names[cls], score * 100),
                fill=color, font=font)

    # converting PIL image back to OpenCV format
    im_np = np.asarray(img)
    return im_np

def load_coco_names(file_name):
    names = {}
    assert os.path.exists(file_name), "path doesn't exist {0}".format(file_name)
    with open(file_name) as f:
        for coco_id, name in enumerate(f):
            names[coco_id] = name
    return names

def letter_box_pos_to_original_pos(letter_pos, current_size,
                                   ori_image_size) -> np.ndarray:
    letter_pos = np.asarray(letter_pos, dtype=np.float)
    current_size = np.asarray(current_size, dtype=np.float)
    ori_image_size = np.asarray(ori_image_size, dtype=np.float)
    final_ratio = min(current_size[0] / ori_image_size[0],
                      current_size[1] / ori_image_size[1])
    pad = 0.5 * (current_size - final_ratio * ori_image_size)
    pad = pad.astype(np.int32)
    to_return_pos = (letter_pos - pad) / final_ratio
    return to_return_pos

def letter_box_image(image_path, input_height, input_width,
                     fill_value) -> np.ndarray:
    image = Image.open(image_path)
    height_ratio = float(input_height) / image.size[1]
    width_ratio = float(input_width) / image.size[0]
    fit_ratio = min(width_ratio, height_ratio)
    fit_height = int(image.size[1] * fit_ratio)
    fit_width = int(image.size[0] * fit_ratio)
    fit_image = np.asarray(
        image.resize((fit_width, fit_height), resample=Image.BILINEAR))

    fill_value = np.full(fit_image.shape[2], fill_value, fit_image.dtype)
    to_return = np.tile(fill_value, (input_height, input_width, 1))
    pad_top = int(0.5 * (input_height - fit_height))
    pad_left = int(0.5 * (input_width - fit_width))
    to_return[pad_top:pad_top + fit_height, pad_left:pad_left +
              fit_width] = fit_image
    return to_return, image

def iou(box1, box2):
    b1_x0, b1_y0, b1_x1, b1_y1 = box1
    b2_x0, b2_y0, b2_x1, b2_y1 = box2

    int_x0 = max(b1_x0, b2_x0)
    int_y0 = max(b1_y0, b2_y0)
    int_x1 = min(b1_x1, b2_x1)
    int_y1 = min(b1_y1, b2_y1)

    int_area = (int_x1 - int_x0) * (int_y1 - int_y0)

    b1_area = (b1_x1 - b1_x0) * (b1_y1 - b1_y0)
    b2_area = (b2_x1 - b2_x0) * (b2_y1 - b2_y0)

    iou = int_area / (b1_area + b2_area - int_area + 1e-05)

    return iou

### Set Variables for Object Detection Sample

In [ ]:
model_file = "data/yolo_v3_160.pb"
input_layer = "inputs"
output_layer = "output_boxes"
label_file = "data/coco.names"
input_height = 160
input_width = 160
input_mean = 0
input_std = 255
conf_threshold = 0.6
iou_threshold = 0.5

### Load the Model, Image, and Labels

In [ ]:
graph = load_graph(model_file)

img_resized, img = letter_box_image(file_name, input_height, input_width, 128)
img_resized = img_resized.astype(np.float32)

labels = load_coco_names(label_file)

### Enable/Disable OpenVINO integration with TensorFlow

Since we already imported OpenVINO-TensorFlow module in the classification sample above, it is already activated. To enable or disable it, you need to use the API calls below accordingly before running the inference.

**To disable** OpenVINO ingtegration with TensorFlow, execute the cell below before running the inference.

In [ ]:
#ovtf.disable()

**To enable** OpenVINO ingtegration with TensorFlow, execute the cell below before running the inference.

In [ ]:
#ovtf.enable()

### Run the Inference

In [ ]:
input_name = "import/" + input_layer
output_name = "import/" + output_layer
input_operation = graph.get_operation_by_name(input_name)
output_operation = graph.get_operation_by_name(output_name)

with tf.compat.v1.Session(graph=graph) as sess:
    # Warmup
    detected_boxes = sess.run(output_operation.outputs[0],
                              {input_operation.outputs[0]: [img_resized]})
    # Run
    import time
    start = time.time()
    detected_boxes = sess.run(output_operation.outputs[0],
                              {input_operation.outputs[0]: [img_resized]})
    elapsed = time.time() - start
    print('Inference time in ms: %f' % (elapsed * 1000))
    
    # apply non max suppresion, draw boxes and show updated image
    filtered_boxes = non_max_suppression(detected_boxes, conf_threshold,
                                         iou_threshold)
    result_img = draw_boxes(filtered_boxes, img, labels, (input_width, input_height), True)
    
    plt.imshow(result_img)
    plt.show()